<a href="https://colab.research.google.com/github/athapa785/LLM_4_Biz_Stanford/blob/main/aditya_thapa_llm4biz_homework_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Initialize key and client

from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')

client = OpenAI(api_key=open_ai_key)

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  # response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are an AI that takes instructions from a human and produces an answer. Be concise in your output."},
    {"role": "user", "content": "Compose a haiku about students studing large language models in Stanford on a Wednesday night."}
  ]
)

print(response)

ChatCompletion(id='chatcmpl-Avwht1xKwl6eCYivm8u0MkAvC4HN2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='In Stanford by night,  \nStudents delve into the code,  \nLanguage models shine.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738373729, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=50, total_tokens=68, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
print(response.choices[0].message.content)

In Stanford by night,  
Students delve into the code,  
Language models shine.


#Homework

> Comparing texts from PDFs, webpages, or user input text.


In [113]:
!pip install beautifulsoup4 requests
!pip install PyPDF2
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.0 MB/s eta 0:00:00


In [114]:
import PyPDF2
from  bs4 import BeautifulSoup
import requests
import tiktoken

In [ ]:
def chat(sys_message, user_message):
    """
    Send a message to the OpenAI GPT-3.5 model and return its response.

    This function interacts with the OpenAI API, specifically using the GPT-3.5-turbo model. It takes a user's message as input, sends it to the model, and returns the model's text-only response. The function ensures the AI's output is concise by providing a system-level instruction.

    Parameters:
    message (str): A string containing the user's message to the AI.

    Returns:
    str: The text response generated by the GPT-3.5 model.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"{sys_message}"},
            {"role": "user", "content": f"{user_message}"}
        ]
    )

    text_only = response.choices[0].message.content
    return text_only

In [135]:
def extract_webpage_text(url):
  """
  Extracts text from a webpage.

  Parameters:
  url (str): The URL of the webpage.

  Returns:
  str: The extracted text from the webpage.
  """
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  text = soup.get_text()
  lines = (line.strip() for line in text.splitlines())
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  text = '\n'.join(chunk for chunk in chunks if chunk)
  return text

In [117]:
def extract_pdf_text(pdf_path):
  """
  Extracts text from a PDF file.

  Parameters:
  pdf_path (str): The path to the PDF file.

  Returns:
  str: The extracted text from the PDF.
  """

  with open(pdf_path, 'rb') as pdf_file:
    pdf_reader = PyPDF.PdfReader(pdf_file)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
      page = pdf_reader.pages[page_num]
      text += page.extract_text()
      lines = (line.strip() for line in text.splitlines())
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

In [118]:
def num_tokens_from_string(string, encoding_name="cl100k_base"):
  """Returns the number of tokens in a text string."""
  encoding = tiktoken.get_encoding(encoding_name)
  num_tokens = len(encoding.encode(string))
  return num_tokens

In [119]:
def summarize(text, max_tokens=2048):
  """
  Summarizes a given text using OpenAI GPT-3.5, limiting the input text size.

  Parameters:
  text (str): The text to be summarized.
  max_tokens (int): The maximum number of tokens allowed for the input text.

  Returns:
  str: The summarized text.
  """
  # Limit the input text size to prevent exceeding token limits
  text_tokens = num_tokens_from_string(text)
  if text_tokens > max_tokens:
    text = text[:max_tokens * 4] # Assume roughly 4 chars per token

  sys_message = f"You are an AI that takes instructions from a human and produces an answer. Be concise in your output."
  user_message = f"Summarize this text: {text}"
  return chat(sys_message, user_message)

In [126]:
def compare_texts(*args):
  """
  Compares two pieces of texts using OpenAI GPT-3.5.

  Parameters:
  text1, text2 (str): Texts to be compared

  """
  summary = []
  i = 1

  for arg in args:
    summary.append(summarize(arg))

  texts = ""
  for text in summary:
    texts += f"({i})" + text + ";"

  user_message = f"Compare {texts} for important differences"

  sys_message = f"You are an AI that takes instructions from a human and produces an answer. Be analytical."
  return chat(sys_message, user_message)

# Let's try out an example.

In [136]:
text1 = extract_webpage_text("https://drlee.io/step-by-step-guide-bayesian-optimization-with-random-forest-fdc6f329db9c")
text2 = extract_pdf_text("waves_quantum.pdf")
text3 = "Quantum computing is going to be the next big thing after LLMs."

In [127]:
compare_texts(text1, text2, text3)

"The first text focuses on Bayesian Optimization as a method to enhance a Random Forest model's performance on a wine quality dataset through hyperparameter tuning, while the second text introduces quantum mechanics, discussing particle behavior based on the Schrodinger equation and key historical figures such as Planck, Einstein, Bohr, and de Broglie. The third text briefly mentions the emergence of quantum computing after LLMs as a significant advancement.\n\nOne important difference is the subject matter: the first text relates to machine learning and optimization techniques for model improvement, the second text delves into quantum mechanics and its historical background, and the third text highlights the potential of quantum computing in comparison to large language models (LLMs).\n\nAnother distinction lies in the focus and application domains. The first text is practical, aiming to enhance model performance on a specific dataset, the second text is theoretical, discussing fundam